## Classical Model Version: XGBoost + Random Forest

- This notebook contains my steps on creating the most optimal model for the classical version of this. 
- The goal for the model is to find the next 5 day return target and to advise whether to buy hold or sell.

Why RBLX? well, I grew up playing roblox and I also have shares invested in roblox lol

### Initial Data Setup

In [172]:
from __future__ import annotations

# ────────────────────── 1. CONFIGURATION ──────────────────────
import os, random, warnings, math, joblib, json, datetime as dt
from pathlib import Path
from typing import Tuple, List

import numpy as np
import pandas as pd
import optuna, ta, holidays
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
import statsmodels.api as sm



In [173]:
# --------------------- User-Editable Hyperparams --------------
DATA_PATH   = "../data/master_dataset_cleaned.csv"   # <- Must contain ≥2021 data
MODEL_DIR   = "../models"
SEED        = 42

HORIZON     = 5          # Days ahead for prediction/log return calculation
TRAIN_WD    = 240        # Training window (~1 year)
TEST_WD     = 40
STEP        = HORIZON    # Walk-forward step (same as prediction horizon)
CONV_PCTL   = 0.25       # Conviction filter quantile
TARGET_VOL  = 0.15       # Target annualized volatility
LEVER_CAP   = 3
BETA_SHRINK = 0.8        # Shrinkage for raw beta estimate
TR_COST     = 0.0002     # Transaction cost (round-trip, 20bp)
BORROW_FEE  = 0.001      # Borrow fee per annum (optional)
RISK_FREE   = 0.00       # Risk-free rate (assume zero for simplicity)

In [174]:
# ─────────────── 2. REPRODUCIBILITY SETUP ───────────────
np.random.seed(SEED)
random.seed(SEED)
warnings.filterwarnings("ignore")
Path(MODEL_DIR).mkdir(parents=True, exist_ok=True)


In [175]:
# ─── 3. LOAD DATA & BUILD TARGET (STRICTLY FORWARD) ────
df = (
    pd.read_csv(DATA_PATH, parse_dates=["date"])    # Expects at least 3 years of data
      .sort_values("date")
      .set_index("date")
)

# ────── CALCULATE 5-DAY LOG RETURN TARGET ('y') ──────
# For each date t, compute log return to t+HORIZON (here, 5 days ahead)
df["y"] = np.log(df["Close"].shift(-HORIZON) / df["Close"])


### Feature Engineering

In [176]:
# ───────────────────── 4. FEATURE ENGINEERING ─────────────────────
LAGS: List[int] = [1, 2, 3, 5, 10, 20]
for l in LAGS:
    df[f"ret_{l}"]     = df["Close"].pct_change(l).shift(1)    # Past returns
    df[f"vol_ret_{l}"] = df["Volume"].pct_change(l).shift(1)   # Past volume change

df["mom_20"] = np.sign(df["Close"].pct_change(20).shift(1))    # 20-day momentum (sign)

# Technical indicators (RSI, ATR, Bollinger Bands)
df["rsi_14"] = ta.momentum.RSIIndicator(df["Close"], 14).rsi().shift(1)
atr = ta.volatility.AverageTrueRange(df["High"], df["Low"], df["Close"], 14)
df["atr_14"] = atr.average_true_range().shift(1)
bb = ta.volatility.BollingerBands(df["Close"], 20)
df["bb_high"], df["bb_low"] = bb.bollinger_hband().shift(1), bb.bollinger_lband().shift(1)

# Lagged macro features
for col in df.columns:
    if col.startswith(("SP500_", "Nasdaq_", "NASDAQ_", "VIX", "vix")):
        df[f"{col}_lag1"] = df[col].shift(1)

# US holidays (binary flag)
us_holidays = holidays.US()
df["is_holiday"] = df.index.to_series().map(lambda d: int(d in us_holidays)).shift(1)

# Forward-fill and set feature start index (to avoid lookahead bias)
df = df.ffill()
feature_cols_prelim = [c for c in df.columns if c != "y"]
first_vals = [df[c].first_valid_index() for c in feature_cols_prelim if df[c].notna().any()]
first_vals = [ts for ts in first_vals if ts is not None]
FEATURE_START = max(first_vals)
df = df.loc[FEATURE_START:].dropna()

In [177]:
# ─── 5. FEATURE SET: FILTER TO SIGNAL-RELEVANT INPUTS ───
DROP_PREFIXES = ("qbs_", "qis_", "qcf_")
RAW_RBLX      = {"Open", "High", "Low", "Close", "Adj Close", "Volume"}
MACRO_SAMEDAY = {c for c in df.columns if c.startswith(("SP500_", "Nasdaq_", "NASDAQ_", "VIX", "vix"))}

feature_cols = [
    c for c in df.columns
    if c != "y" and not c.startswith(DROP_PREFIXES)
    and c not in RAW_RBLX and c not in MACRO_SAMEDAY
]

print(f"[INFO] Final feature count: {len(feature_cols)}")

[INFO] Final feature count: 32


### Training

In [178]:
# ───── 6. HYPERPARAMETER TUNING WITH OPTUNA ─────
ts_train = df.iloc[:TRAIN_WD]
X_tune, y_tune = ts_train[feature_cols].values, ts_train["y"].values
tscv = TimeSeriesSplit(n_splits=3, test_size=TEST_WD, gap=HORIZON)

def objective(trial: optuna.trial.Trial) -> float:
    """Objective function for Optuna hyperparameter search using time series split."""
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 400, 1200),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.03, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 10),
        "min_child_weight": trial.suggest_float("min_child_weight", 0, 5),
        "reg_lambda": trial.suggest_float("reg_lambda", 0, 10),
        "objective": "reg:squarederror",
        "tree_method": "hist",
        "device": "cuda",
        "random_state": SEED,
        "early_stopping_rounds": 30,
    }
    scores = []
    for tr_idx, val_idx in tscv.split(X_tune):
        split = int(len(tr_idx) * 0.9)
        X_fit, y_fit = X_tune[tr_idx][:split], y_tune[tr_idx][:split]
        X_es,  y_es  = X_tune[tr_idx][split:], y_tune[tr_idx][split:]
        mdl = XGBRegressor(**params, verbosity=0)
        mdl.fit(X_fit, y_fit, eval_set=[(X_es, y_es)], verbose=False)
        y_pred = mdl.predict(X_tune[val_idx])
        scores.append(mean_absolute_error(y_tune[val_idx], y_pred))
    return float(np.mean(scores))

print("🔍  Optuna hyper‑param search … (≤ 60 trials or 1 h)")
study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=SEED))
study.optimize(objective, n_trials=60, timeout=3600, show_progress_bar=True)
params_best = study.best_trial.params
joblib.dump(params_best, f"{MODEL_DIR}/best_params.pkl")


[I 2025-07-18 13:11:18,133] A new study created in memory with name: no-name-8cfcac31-3122-4918-ad0f-0b6498b9e9e5


🔍  Optuna hyper‑param search … (≤ 60 trials or 1 h)


  0%|          | 0/60 [00:00<?, ?it/s]

[I 2025-07-18 13:11:18,513] Trial 0 finished with value: 0.05536593186013172 and parameters: {'n_estimators': 700, 'max_depth': 10, 'learning_rate': 0.1618509290001068, 'subsample': 0.8394633936788146, 'colsample_bytree': 0.6624074561769746, 'gamma': 1.5599452033620265, 'min_child_weight': 0.2904180608409973, 'reg_lambda': 8.661761457749352}. Best is trial 0 with value: 0.05536593186013172.
[I 2025-07-18 13:11:18,831] Trial 1 finished with value: 0.05543984810469205 and parameters: {'n_estimators': 881, 'max_depth': 8, 'learning_rate': 0.031456163175583855, 'subsample': 0.9879639408647978, 'colsample_bytree': 0.9329770563201687, 'gamma': 2.1233911067827616, 'min_child_weight': 0.9091248360355031, 'reg_lambda': 1.8340450985343382}. Best is trial 0 with value: 0.05536593186013172.
[I 2025-07-18 13:11:19,146] Trial 2 finished with value: 0.055364926653338274 and parameters: {'n_estimators': 643, 'max_depth': 7, 'learning_rate': 0.08110848199986004, 'subsample': 0.7164916560792167, 'colsam

['../models/best_params.pkl']

In [179]:
# ─── 7. WALK-FORWARD LOOP (β-NEUTRAL, VOL-TARGETED) ───
target_5d_vol = TARGET_VOL / math.sqrt(252 / HORIZON)
records: List[Tuple[dt.date, float, float, float]] = []

bench_col = next(
    (c for c in df.columns if any(k in c.lower() for k in ["sp500", "nasdaq", "ndx", "spx"]) and "_lag1" not in c.lower()),
    None
)

for start in range(0, len(df) - TRAIN_WD - TEST_WD + 1, STEP):
    train = df.iloc[start: start + TRAIN_WD]
    test  = df.iloc[start + TRAIN_WD: start + TRAIN_WD + TEST_WD]

    X_tr, y_tr = train[feature_cols].values, train["y"].values
    model = XGBRegressor(**params_best, objective="reg:squarederror", tree_method="hist", device="cuda", random_state=SEED)
    model.fit(X_tr, y_tr)

    y_hat = model.predict(test[feature_cols].values)
    y_hat_train = model.predict(X_tr)  # For quantile-based filter

    # --- STRONGER CONVICTION FILTER (Quantile, Top/Bottom 5%) ---
    PCTL = 0.05
    thr = np.quantile(np.abs(y_hat_train), PCTL)
    raw_pos = np.sign(y_hat)
    raw_pos[np.abs(y_hat) < thr] = 0

    # --- β-HEDGE (market neutrality) ---
    if bench_col:
        r_bmk = np.log(train[bench_col].shift(-HORIZON) / train[bench_col]).dropna()
        r_ast = np.log(train["Close"].shift(-HORIZON) / train["Close"]).dropna()
        beta_hat = (np.cov(r_bmk.align(r_ast)[0], r_ast.align(r_bmk)[0])[0, 1] / r_bmk.var()) if r_bmk.var() else 0.0
    else:
        beta_hat = 0.0
    beta_hat *= BETA_SHRINK

    excess_hist = (r_ast - beta_hat * r_bmk) if bench_col else r_ast
    sigma_5d    = excess_hist.std(ddof=1)
    scale       = target_5d_vol / sigma_5d if sigma_5d > 0 else 1.0

    # --- POSITION SIZING & ENFORCE EXIT ---
    pos_scaled  = np.clip(raw_pos * scale, -LEVER_CAP, LEVER_CAP)
    enter_idx = test.index[::HORIZON]
    exit_idx  = enter_idx + pd.Timedelta(days=HORIZON)
    pos_enter = pd.Series(pos_scaled[::HORIZON], index=enter_idx)
    pos_exit  = pd.Series(0.0, index=exit_idx)
    positions = (
        pd.concat([pos_enter, pos_exit])
          .sort_index()
          .reindex(test.index, method="ffill")
          .fillna(0.0)
    )

    # --- RETURNS CALCULATION ---
    asset_ret   = np.log(test["Close"].shift(-HORIZON) / test["Close"])
    bmk_ret     = (np.log(test[bench_col].shift(-HORIZON) / test[bench_col]) if bench_col else pd.Series(0.0, index=asset_ret.index))
    excess_ret  = asset_ret - beta_hat * bmk_ret

    records.extend(list(zip(asset_ret.index, positions.values, excess_ret.values, bmk_ret.values)))

# --- Build DataFrame with Results ---
pred_df = pd.DataFrame(records, columns=["date", "pos", "excess_ret", "bmk_ret"]).set_index("date")

# --- Turnover & Transaction Cost Calculation ---
turnover_series = pred_df["pos"].diff().abs()
turnover_series.iloc[0] = pred_df["pos"].abs().iloc[0]
turnover = turnover_series.mean()
trade_count = (turnover_series > 0).sum()
fric = TR_COST * turnover_series
strat_ret = pred_df["pos"] * pred_df["excess_ret"] - fric

### Analysis

In [180]:
# ────────────── 8. METRICS AND BOOTSTRAP ──────────────
ann_factor = 252 / HORIZON
geom_ret = np.exp(np.log1p(strat_ret).sum() * ann_factor / len(strat_ret)) - 1
ann_mu   = strat_ret.mean() * ann_factor
ann_vol  = strat_ret.std(ddof=1) * math.sqrt(ann_factor)
sharpe   = (ann_mu - RISK_FREE) / ann_vol if ann_vol else np.nan

# Downside volatility & Sortino ratio
neg = strat_ret[strat_ret < 0]
sortino = (ann_mu - RISK_FREE) / (neg.std(ddof=1) * math.sqrt(ann_factor)) if len(neg) else np.nan

# Drawdown & Calmar ratio
cum_eq = (1 + strat_ret).cumprod()
rolling_max = cum_eq.cummax()
dd = cum_eq / rolling_max - 1
max_dd = dd.min()
calmar = ann_mu / abs(max_dd) if max_dd < 0 else np.nan

# Information ratio (vs benchmark)
excess = strat_ret - pred_df["bmk_ret"]
ir = excess.mean() / excess.std(ddof=1) if excess.std(ddof=1) else np.nan

# Directional accuracy
hit = (np.sign(pred_df["pos"] * pred_df["excess_ret"]) > 0).mean()

In [181]:
# --- Bootstrapped confidence intervals for metrics ---
def bootstrap_metric(metric_fn, returns, n_boot=1000):
    returns = np.asarray(returns[~np.isnan(returns)])
    stats = []
    for _ in range(n_boot):
        sample = np.random.choice(returns, size=len(returns), replace=True)
        val = metric_fn(sample)
        if not np.isnan(val) and np.isfinite(val):
            stats.append(val)
    return np.mean(stats), np.std(stats, ddof=1)

def sharpe_fn(ret):
    m = np.mean(ret) * ann_factor
    v = np.std(ret, ddof=1) * np.sqrt(ann_factor)
    return (m - RISK_FREE) / v if v > 0 else np.nan

def sortino_fn(ret):
    m = np.mean(ret) * ann_factor
    neg = ret[ret < 0]
    v = np.std(neg, ddof=1) * np.sqrt(ann_factor)
    return (m - RISK_FREE) / v if v > 0 else np.nan

def calmar_fn(ret):
    ret = pd.Series(ret)
    eq = (1 + ret).cumprod()
    rolling_max = eq.cummax()
    dd = eq / rolling_max - 1
    mdd = dd.min()
    mu = np.mean(ret) * ann_factor
    return mu / abs(mdd) if mdd < 0 else np.nan

def hit_fn(ret):
    return (np.sign(ret) > 0).mean()

# --- Moving-block bootstrap for Sharpe ratio ---
def block_bootstrap(returns, block_size, n_boot=1000):
    N = len(returns)
    stats = []
    for _ in range(n_boot):
        idx = np.random.randint(0, N - block_size + 1, N // block_size)
        sample = np.concatenate([returns[i:i+block_size] for i in idx])
        stats.append(sharpe_fn(sample[:N]))
    return np.array(stats)

In [182]:
sharpe_mu, sharpe_se = bootstrap_metric(sharpe_fn, strat_ret.values)
sortino_mu, sortino_se = bootstrap_metric(sortino_fn, strat_ret.values)
calmar_mu, calmar_se = bootstrap_metric(calmar_fn, strat_ret.values)
hit_mu, hit_se = bootstrap_metric(hit_fn, (pred_df["pos"] * pred_df["excess_ret"]).values)

clean_ret = strat_ret.dropna().values
BLOCK = int(np.sqrt(len(clean_ret)))
sharpe_dist = block_bootstrap(clean_ret, block_size=BLOCK)
sharpe_dist = sharpe_dist[np.isfinite(sharpe_dist)]
reality_p = (sharpe_dist > 3).mean()

# --- Residual β and CI via OLS regression ---
mask_beta = pred_df["bmk_ret"].abs() > 0
Xb = pred_df.loc[mask_beta, "bmk_ret"]
yb = strat_ret[mask_beta]
if len(Xb) > 1 and Xb.var() > 0:
    Xc = sm.add_constant(Xb)
    model = sm.OLS(yb, Xc).fit()
    resid_beta = model.params[1]
    resid_beta_ci = model.conf_int().loc["bmk_ret"].values
else:
    resid_beta, resid_beta_ci = np.nan, (np.nan, np.nan)

# --- Pesaran-Timmermann p-value (significance of direction accuracy) ---
try:
    import scipy.stats as sts
    n = len(pred_df)
    pt_num = (hit - 0.5) * math.sqrt(n)
    pt_p = 2 * (1 - sts.norm.cdf(abs(pt_num)))
except Exception:
    pt_p = np.nan

In [183]:
# --- PRINT SUMMARY ---
print("================ WALK-FORWARD SUMMARY ================")
print(f"Trades                  : {trade_count}")
if trade_count < 20:
    print("⚠️  WARNING: < 20 trades — all performance stats are extremely noisy and unreliable.")
print(f"Geometric ann. return   : {geom_ret:.2%}")
print(f"Annual volatility       : {ann_vol:.2%}")
print(f"Sharpe ratio            : {sharpe:.2f} ± {sharpe_se:.2f}")
print(f"Sortino ratio           : {sortino:.2f} ± {sortino_se:.2f}")
print(f"Calmar ratio            : {calmar:.2f} ± {calmar_se:.2f}  (max DD {max_dd:.2%})")
print(f"Information ratio (μ/σ) : {ir:.2f}")
print(f"Direction accuracy      : {hit:.2%} ± {hit_se:.2%}  (PT p={pt_p:.3g})")
print(f"Turnover                : {turnover:.2f}")
print(f"Residual beta           : {resid_beta:.2f}   CI: [{resid_beta_ci[0]:.2f}, {resid_beta_ci[1]:.2f}]")
print(f"Block-bootstrap: Sharpe > 3 in {reality_p*100:.1f}% of bootstraps.")

# --- SAVE METRICS FOR AUDIT TRAIL ---
metrics = {
    "geom_ret": float(geom_ret),
    "ann_vol": float(ann_vol),
    "sharpe": float(sharpe), "sharpe_se": float(sharpe_se),
    "sortino": float(sortino), "sortino_se": float(sortino_se),
    "calmar": float(calmar), "calmar_se": float(calmar_se),
    "ir": float(ir),
    "hit": float(hit), "hit_se": float(hit_se),
    "turnover": float(turnover),
    "resid_beta": float(resid_beta),
    "resid_beta_ci": [float(x) for x in resid_beta_ci],
    "trade_count": int(trade_count),
    "max_dd": float(max_dd),
    "sharpe_bootstrap_dist": sharpe_dist.tolist(),
    "reality_sharpe_gt3_pct": float(reality_p),
}
with open(f"{MODEL_DIR}/walkforward_metrics.json", "w") as fp:
    json.dump(metrics, fp, indent=2)

================ WALK-FORWARD SUMMARY ================
Trades                  : 124
Geometric ann. return   : 62.79%
Annual volatility       : 9.88%
Sharpe ratio            : 5.72 ± 0.49
Sortino ratio           : 9.84 ± 2.62
Calmar ratio            : 8.99 ± 4.22  (max DD -6.28%)
Information ratio (μ/σ) : 0.01
Direction accuracy      : 60.56% ± 2.61%  (PT p=0.0452)
Turnover                : 0.12
Residual beta           : 0.09   CI: [0.04, 0.14]
Block-bootstrap: Sharpe > 3 in 100.0% of bootstraps.


In [184]:
# ───────────── 9. STRESS TESTS ─────────────
print("\n[STRESS‑TESTS]")
bear_mask = (pred_df.index >= "2022-04-01") & (pred_df.index <= "2022-06-30")
if bear_mask.any():
    bear_acc = (np.sign(pred_df.loc[bear_mask, "pos"] * pred_df.loc[bear_mask, "excess_ret"]) > 0).mean()
    print(f"DirAcc 2022‑Q2 bear   : {bear_acc:.2%}")

if {"Open", "Close"}.issubset(df.columns):
    gap = np.abs((df["Open"] - df["Close"].shift(1)) / df["Close"].shift(1))
    gap_days = gap.loc[pred_df.index] > 0.02
    if gap_days.any():
        gap_acc = (np.sign(pred_df.loc[gap_days, "pos"] * pred_df.loc[gap_days, "excess_ret"]) > 0).mean()
        print(f"DirAcc gap >2%        : {gap_acc:.2%}")

atr20 = df["atr_14"].quantile(0.2)
low_vol = (df["atr_14"] < atr20).loc[pred_df.index]
if low_vol.any():
    low_acc = (np.sign(pred_df.loc[low_vol, "pos"] * pred_df.loc[low_vol, "excess_ret"]) > 0).mean()
    print(f"DirAcc low‑ATR        : {low_acc:.2%}")



[STRESS‑TESTS]
DirAcc gap >2%        : 57.33%


In [185]:
# ───────────── 10. PLOT EQUITY CURVE ─────────────
plt.style.use("dark_background")
fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(cum_eq.index, cum_eq - 1, label="Strategy")
ax.plot((1 + pred_df["bmk_ret"]).cumprod().index, (1 + pred_df["bmk_ret"]).cumprod() - 1, label="Benchmark")
ax.set_title("Leak‑Free Walk‑Forward Back‑test (Net of Friction)")
ax.legend()
fig.tight_layout()
plt.savefig(f"{MODEL_DIR}/equity_curve.png", dpi=150)
plt.close()


### Model Saving

In [186]:
# ───────────── 11. SAVE FINAL MODEL ─────────────
full_X, full_y = df[feature_cols].values, df["y"].values
final_model = XGBRegressor(**params_best, objective="reg:squarederror", tree_method="hist", device="cuda", random_state=SEED)
final_model.fit(full_X, full_y)
joblib.dump(final_model, f"{MODEL_DIR}/rblx_xgb_5d_walkforward.pkl")
print(f"\n✅  Final model & metrics stored in '{MODEL_DIR}/'\n")


✅  Final model & metrics stored in '../models/'

